Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier

Загружаем данные

In [2]:
transactions_train = pd.read_csv('transactions_train.csv')
train_target = pd.read_csv('train_target.csv')
transactions_test = pd.read_csv('transactions_test.csv')
test_id = pd.read_csv('test.csv')
categories = pd.read_excel('small_group_description.xlsx')[['name', 'code']]

transactions_train и transactions_test хранят информацию о транзакциях в формате клиент-дата-стоимость-категория

train_target - это информация про каждого клиента о том была ли совершена в этих категориях покупка на следующей неделе

test_id - это клиенты для которых надо предсказать совершат ли они покупку в тех категориях на следующей неделе

categories - это описание категорий

In [3]:
transactions_train.head()

,client_dk,trans_date,amount,small_group
0,43976,0,4.563,2
1,8417,0,48.342,0
2,17309,0,12.320,0
3,33523,0,29.005,6
4,24228,0,10.266,6


In [4]:
train_target.head()

,client_dk,27,32,41,45,67,73,81,88
0,39762,1,0,0,0,0,0,0,0
1,10586,0,0,0,1,0,0,0,0
2,40115,0,1,0,0,0,0,0,0
3,34543,0,0,0,1,0,0,0,0
4,5372,0,0,0,1,0,0,0,0


In [5]:
transactions_test.head()

,client_dk,trans_date,amount,small_group
0,31453,0,20.533,0
1,3615,0,7.700,0
2,19950,0,13.441,1
3,7311,0,8.675,7
4,36112,0,23.459,2


In [6]:
test_id.head()

,client_dk
0,12671
1,14015
2,1500
3,4934
4,11405


In [7]:
categories.head()

,name,code
0,Оплата телефона и связи,0
1,Каршеринг или аренда автотранспорта,1
2,Такси и каршеринги,2
3,Несетевые супермаркеты и продуктовые магазины,3
4,Метро (паромы и Ж\Д аналоги),4


Функции для выделения фичей по сумме покупок и количеству покупок в каждой категории за определенный период

In [8]:
def get_amount_features(transactions, prefix):
    amount_cat = transactions.groupby(['client_dk','small_group']).sum()['amount']
    amount_cat = amount_cat.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount').fillna(0)
    amount_cat.columns = [prefix + str(i) for i in amount_cat.columns]
    return amount_cat

In [9]:
def get_count_features(transactions, prefix):
    counter_df = transactions.groupby(['client_dk','small_group'])['amount'].count()
    cat_counts = counter_df.reset_index().pivot(index='client_dk', \
                                                      columns='small_group',values='amount').fillna(0)
    cat_counts.columns = [prefix + str(i) for i in cat_counts.columns]
    return cat_counts

Функция выделяет фичи по сумме покупок и количеству покупок в каждой категории для:

1) Всех транзакций

2) Нормированных транзакций(суммой больше 100 и меньше 1500), чтобы избежать выбросов

3) Транзакций за последнюю неделю

4) Транзакций за последние 3 дня

5) Транзакций за последние две недели

6) Транзакций за последний месяц

7) Транзакций за спортивный пик(то есть время когда было больше всего транзакций для спортивных матчей(см. analytics.ipynb))

8) Транзакций за цветочный пик(то есть время когда было больше всего транзакций для цветов(см. analytics.ipynb))

9) Транзакций за неделю годом ранее(возможно у людей ежегодная традиция или какой-то праздник)

In [10]:
def get_features(transactions, target):
    agg_features = transactions.groupby('client_dk')['amount'].agg(['mean','max','min','std','sum','count']).reset_index()
    
    count_cat = get_count_features(transactions, 'cat_count_')
    amount_cat = get_amount_features(transactions, 'cat_spend_')
    
    count_cat_norm = get_count_features(transactions[(transactions.amount >= 100) & (transactions.amount <= 1500)], 'cat_count_norm_')
    amount_cat_norm = get_amount_features(transactions[(transactions.amount >= 100) & (transactions.amount <= 1500)], 'cat_spend_norm_')
    
    amount_cat_last_week = get_amount_features(transactions[transactions.trans_date > 380], 'cat_spend_last_week_')
    count_cat_last_week = get_count_features(transactions[transactions.trans_date > 380], 'cat_count_last_week_')  
    
    amount_cat_last_3days = get_amount_features(transactions[transactions.trans_date > 384], 'cat_spend_last_3days_')
    count_cat_last_3days = get_count_features(transactions[transactions.trans_date > 384], 'cat_count_last_3days_')  
    
    amount_cat_last_2weeks = get_amount_features(transactions[transactions.trans_date > 373], 'cat_spend_last_2weeks_')
    count_cat_last_2weeks = get_count_features(transactions[transactions.trans_date > 373], 'cat_count_last_2weeks_')   
    
    amount_cat_last_month = get_amount_features(transactions[transactions.trans_date > 357], 'cat_spend_last_month_')
    count_cat_last_month = get_count_features(transactions[transactions.trans_date > 357], 'cat_count_last_month_')    
    
    amount_cat_sport_peak = get_amount_features(transactions[(transactions.trans_date >= 283) & (transactions.trans_date <= 300)], 'cat_spend_sport_peak_')
    count_cat_sport_peak = get_count_features(transactions[(transactions.trans_date >= 283) & (transactions.trans_date <= 300)], 'cat_count_sport_peak_')    
    
    amount_cat_flower_peak = get_amount_features(transactions[(transactions.trans_date >= 126) & (transactions.trans_date <= 127)], 'cat_spend_flower_peak_')
    count_cat_flower_peak = get_count_features(transactions[(transactions.trans_date >= 126) & (transactions.trans_date <= 127)], 'cat_count_flower_peak_')    
    
    amount_cat_week_year_ago = get_amount_features(transactions[(transactions.trans_date >= 333) & (transactions.trans_date <= 339)], 'cat_spend_week_year_ago_')
    count_cat_week_year_ago = get_count_features(transactions[(transactions.trans_date >= 333) & (transactions.trans_date <= 339)], 'cat_count_week_year_ago_')   
    
    out = pd.merge(target, agg_features, on='client_dk', how='outer')
    out = pd.merge(out, count_cat.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, amount_cat.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, count_cat_last_week.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, amount_cat_last_week.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, count_cat_last_3days.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, amount_cat_last_3days.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, count_cat_last_2weeks.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, amount_cat_last_2weeks.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, count_cat_last_month.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, amount_cat_last_month.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, count_cat_sport_peak.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, amount_cat_sport_peak.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, count_cat_flower_peak.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, amount_cat_flower_peak.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, count_cat_week_year_ago.reset_index(), on='client_dk', how='outer')
    out = pd.merge(out, amount_cat_week_year_ago.reset_index(), on='client_dk', how='outer')
    
    return out

Генерируем фичи для тренировочного и тестового датасета

In [11]:
train = get_features(transactions_train, train_target)

In [12]:
test = get_features(transactions_test, test_id[['client_dk']])

Находим общие фичи, так как возможно некоторые сгенерировались только в тренировочном или только в тестовом

и затем убираем все категории с номером большим 99, так как частота покупок там слишком редкая и значит нерепрезентативная

In [13]:
common_features = list(set(test.columns).intersection(set(train.columns)))
cf = []
for i in common_features:
    if "cat" in i:
        if int(i.split("_")[-1]) < 100:
            cf.append(i)
    else:
        cf.append(i)
common_features = cf

Создаём тестовый и тренировочный датасет только с нужными фичами

In [14]:
X_train = train[common_features]
X_test = test[common_features]

Устанавливаем параметры для CatBoostClassifier, подобранные тестовым путём

In [15]:
params = {
    'n_estimators': 1000,
    'depth': 6,
    'random_state':42,
    'learning_rate': 0.027,
    'eval_metric': 'AUC',
    'loss_function': 'MultiClass',
    'verbose': 1000, 
    
}

Создаём 8 моделей для каждой категории, которые предсказывают от 0 до 1, будет ли покупка в данной категории

In [16]:
results_tree = {}
main_result = [] 
for q in train_target.columns[1:]:
    print('Training model for category ' + str(q))
    curr_target_train = train_target.loc[:,q]
    model = CatBoostClassifier(**params)
    model.fit(X_train.fillna(0).values, curr_target_train.values)
    main_result.append(model.get_evals_result())
    pred = model.predict_proba(X_test.fillna(0).values)[:,1]
    results_tree[q] = pred

Training model for category 27
0:	total: 362ms	remaining: 6m 1s
999:	total: 5m 28s	remaining: 0us
Training model for category 32
0:	total: 359ms	remaining: 5m 58s
999:	total: 5m 12s	remaining: 0us
Training model for category 41
0:	total: 422ms	remaining: 7m 1s
999:	total: 5m 27s	remaining: 0us
Training model for category 45
0:	total: 357ms	remaining: 5m 57s
999:	total: 5m 49s	remaining: 0us
Training model for category 67
0:	total: 318ms	remaining: 5m 17s
999:	total: 5m 18s	remaining: 0us
Training model for category 73
0:	total: 327ms	remaining: 5m 26s
999:	total: 5m 18s	remaining: 0us
Training model for category 81
0:	total: 314ms	remaining: 5m 13s
999:	total: 4m 57s	remaining: 0us
Training model for category 88
0:	total: 311ms	remaining: 5m 10s
999:	total: 4m 59s	remaining: 0us


### Подготовим файл для отправки в систему

In [17]:
submission = pd.DataFrame(data=np.zeros((25000,8)), columns=train_target.columns[1:], index=test_id['client_dk'].values)

In [18]:
for q in results_tree:
    submission[q] = results_tree[q]
submission.columns = ['cat_27','cat_32','cat_41','cat_45','cat_67','cat_73','cat_81','cat_88']

In [19]:
submission.index.name = 'client_dk'

Сохраняем прогноз на диск в папку submissions. Имя прогноза соответсвует дате и времени его создания, закодированными с помощью timestamp.

In [20]:
import time
import os

current_timestamp = int(time.time())
submission_path = 'submissions/{}.csv'.format(current_timestamp)

if not os.path.exists('submissions'):
    os.makedirs('submissions')

submission.to_csv(submission_path, index=True)